<div style="background-color:#1e1e1e; padding:25px; border-radius:10px; display:table; width:100%;">

  <!-- Left column: text -->
  <div style="display:table-cell; vertical-align:top; width:70%; padding-right:10px;">
    <h1 style="color:#4FC3F7; margin-bottom:10px;">Hands-On Part B: Personalization Methods</h1>

  <p style="font-size:18px; font-style:italic; color:#cccccc;">
    Workshop on T1D Simulator and Digital Twins for Personalized Care
  </p>

  <p style="font-size:15px; line-height:1.6; color:#dddddd;">
fill
  </p>
  </div>

  <!-- Right column: logo -->
  <div style="display:table-cell; vertical-align:middle; text-align:left; width:30%; padding-right:50px;">
    <img src="https://micelab.udg.edu/wp-content/uploads/2022/08/MICElab-letras_png-300x119.png" alt="MiceLab Logo" style="height:80px; border-radius:8px;">
  </div>

</div>


<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
0. Imports &amp; Set-Up
</h2>

In [1]:
from pathlib import Path

import numpy as np
import tensorflow as tf
import pandas as pd

from sklearn.model_selection import train_test_split

from data_classes import TrainingData
from gan import GANModel

2025-09-18 20:24:07.221612: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-18 20:24:07.223038: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-18 20:24:07.257092: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-18 20:24:07.258856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-18 20:24:07.962140: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Check GPU availability for TensorFlow, not essential.

In [2]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPUs:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.13.0
Num GPUs Available: 0
GPUs: []


<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
1. Load Model & Data
</h2>

Setup path and model name.

In [3]:
path_to_model = "/mnt/c/Users/oriol/Downloads/"
model_name = "pretrained_model.h5"
model_name = "decoder_epoch_10.h5"

Load the .h5 using the `load_model` method from keras.

In [4]:
model_filename = Path(path_to_model, model_name)
pretrained_vae_model = tf.keras.models.load_model(model_filename)

In [5]:
path_df = Path("data_processed.csv")

df = pd.read_csv(path_df)

display(df.head())

,ID,BG,PI,RA
0,11_0,134.0,0.000000,0.0
1,11_0,125.0,0.000271,0.0
2,11_0,132.0,0.001510,0.0
3,11_0,132.0,0.003488,0.0
4,11_0,132.0,0.006741,0.0


<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
2. Split Patient Data for Train / Validation
</h2>

In [6]:
split_size = 0.2
df_train, df_val = train_test_split(df, test_size=split_size, random_state=0, shuffle=True)

display(df_train.head())
print("Train shape:", df_train.shape)
print("Validation shape:", df_val.shape)

,ID,BG,PI,RA
1649,290_0,166.0,1.851760,0.231335
1465,290_0,63.0,0.834074,0.018890
361,11_0,109.0,0.436116,0.000001
315,11_0,93.0,0.275721,0.000246
1372,290_0,131.0,0.855476,0.032081


Train shape: (1382, 4)
Validation shape: (346, 4)


In [7]:
# scale columns
numeric_cols = ['BG', 'PI', 'RA']
df_train_mean = df_train[numeric_cols].mean()
df_train_std = df_train[numeric_cols].std()
df_train_scaled = df_train.copy()
df_train_scaled[numeric_cols] = (df_train[numeric_cols] - df_train_mean) / df_train_std

# rename columns
new_column_names = {col: f'sc{col}' for col in numeric_cols}
df_train_scaled = df_train_scaled.rename(columns=new_column_names)
df_train_scaled

,ID,scBG,scPI,scRA
1649,290_0,1.209841,1.803236,-0.322233
1465,290_0,-1.116711,-0.214161,-0.525076
361,11_0,-0.077668,-1.003046,-0.543111
315,11_0,-0.439075,-1.321004,-0.542877
1372,290_0,0.419265,-0.171735,-0.512480
...,...,...,...,...
835,11_0,-0.190608,-0.452106,-0.070762
1216,290_0,1.842302,0.203086,-0.543038
1653,290_0,0.374089,1.665554,-0.393609
559,11_0,-0.597190,-1.009164,-0.157899


In [8]:
gan_inputs = ['BG', 'PI', 'RA']

In [9]:
def load_training_data():
    train_data_dict = {}
    for input_name in gan_inputs:
        train_data_dict[f'sc{input_name}'] = df_train_scaled[f'sc{input_name}']

    train_data = TrainingData(**train_data_dict)
    return train_data

In [10]:
training_data = load_training_data()

In [12]:
gan = GANModel(pretrained_vae_model)

try:
    gan.train(training_data)
except KeyboardInterrupt:
    print('Process interrupted by user')

GAN Training:   0%|          | 0/2150 [12:38<?, ?step/s]


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Index out of range using input dim 1; input has only 1 dims [Op:StridedSlice] name: strided_slice/